## Extract the Kinematics

This notebook is used to extract the Kinematics given a trained model. 
Change checkpoint to the model you trained

In [6]:
from src.inference.predict import predict_kinematics
from src.eval.postprocess import postprocess_kinematics
from src.safety.filters import validate_trajectory
from src.safety.dvrk_interface import DVRKInterface
import torch
import cv2
import numpy as np
from src.training.train_vit_system import EEGInformedViTModel
from src.data.transforms_vit import get_vit_transforms



In [7]:
# 1. Load model
checkpoint = torch.load('checkpoints/suturing_fold1_gripper/checkpoint_epoch_10.pth') #checkpoints/baseline/checkpoint_epoch_10.pth
model = EEGInformedViTModel(checkpoint['config'])
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

# 2. Load and process video
cap = cv2.VideoCapture('Gestures/Suturing/video/Suturing_C002_capture2.avi') #Gestures/Needle_Passing/video/Needle_Passing_B001_capture1.avi
frames = []
while True:
    ret, frame = cap.read()
    if not ret:
        break
    frames.append(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
cap.release()

# Apply preprocessing transforms (resize to 224x224 and normalize)
transforms = get_vit_transforms(mode='val', target_size=224, normalize=True)
frames_array = np.array(frames)  # Shape: (T, H, W, C)
frames_tensor = transforms(frames_array)  # Shape: (T, C, 224, 224)


#frames_tensor = torch.from_numpy(np.array(frames)).permute(0, 3, 1, 2).float() / 255.0

# 3. Predict kinematics
with torch.no_grad():
    kinematics = predict_kinematics(model, frames_tensor, postprocess=True)

# 4. Post-process for smoothness
kinematics_smooth = postprocess_kinematics(kinematics)

Initializing EEGInformedViTModel
Creating visual encoder: vit_small_patch16_224
  - Pretrained: True
  - Freeze until layer: 6
  - Use adapters: False
Flow encoder: disabled
Creating temporal aggregator...
  - d_model: 384, n_heads: 6, num_layers: 4
Creating kinematics module...
  - d_kin_input: 19, d_kin_output: 19
  - num_gestures: 15, num_skills: 3
Brain RDM: disabled

Model Parameters:
  - Total: 44,975,141 (44.98M)
  - Trainable: 33,957,413 (33.96M)


In [8]:
# kinematics tensor:
kinematics_smooth.shape

torch.Size([3280, 19])

In [9]:
kinematics_smooth_tensor = kinematics_smooth.cpu().numpy()

# save kinematics tensor
np.save('Kinematics_pred/Needle_Passing/kinematics_tensor_psm2_suturing_final_gripper.npy', kinematics_smooth_tensor)